# Introduction

This  notebook is meant to be a starting point for teams competing in Bot Xchange in ShriTeq 2022. It will help you:

- Get started with processing the data

- Learn how to reshape the time series data into something you can use for your mode

- Get started with a rudimentary RandomForestRegressor model to predict stock prices

To use this notebook:

1. Click the three dots at the top right of this page

undefined. Click 'Download'. This will download this notebook and all project files to your local filesystem

# Data processing

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('opening_prices_biotech.csv')
df.head()

,ticker,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,...,day_941,day_942,day_943,day_944,day_945,day_946,day_947,day_948,day_949,day_950
0,NVO,49.0179,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,49.8548,50.01,...,48.9641,49.0019,50.4401,49.9102,49.9291,49.7021,48.9735,48.7938,48.2923,47.4123
1,VRTX,121.0900,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,123.0300,123.89,...,174.4800,177.0100,178.2300,176.3200,172.3100,172.0500,170.2900,169.5500,169.2400,167.6900
2,REGN,535.5900,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,531.7600,519.38,...,284.6600,288.7500,297.3600,295.0200,290.2100,287.7000,280.4500,275.3000,277.2700,275.3100
3,SGEN,42.3100,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,43.8800,42.07,...,73.0200,73.1500,73.8100,75.0700,73.0800,73.8200,76.7000,79.1700,85.6000,85.2600
4,ALNY,88.2700,87.2400,91.2900,90.1600,92.4900,93.4800,94.1800,92.1800,93.49,...,85.8400,84.9000,86.6900,85.1800,83.0400,83.6300,80.1300,81.1000,80.0700,74.1500


For our model that predicts the next opening price of a stock, we'll take the last _n_ days' prices as input data. We will set n = 50 for this notebook, but **we encourage you to experiment** with what number of features work. 

We will also take all-time historical data, such as the standard deviation, historical highs and lows, as well as the mean value of the stock.

In [3]:
# Only do this if you're using sets of 50
df.drop('day_0', axis=1, inplace=True)
df.head()

,ticker,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,...,day_941,day_942,day_943,day_944,day_945,day_946,day_947,day_948,day_949,day_950
0,NVO,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,49.8548,50.01,49.2164,...,48.9641,49.0019,50.4401,49.9102,49.9291,49.7021,48.9735,48.7938,48.2923,47.4123
1,VRTX,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,123.0300,123.89,120.8000,...,174.4800,177.0100,178.2300,176.3200,172.3100,172.0500,170.2900,169.5500,169.2400,167.6900
2,REGN,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,531.7600,519.38,499.0000,...,284.6600,288.7500,297.3600,295.0200,290.2100,287.7000,280.4500,275.3000,277.2700,275.3100
3,SGEN,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,43.8800,42.07,41.8400,...,73.0200,73.1500,73.8100,75.0700,73.0800,73.8200,76.7000,79.1700,85.6000,85.2600
4,ALNY,87.2400,91.2900,90.1600,92.4900,93.4800,94.1800,92.1800,93.49,92.7600,...,85.8400,84.9000,86.6900,85.1800,83.0400,83.6300,80.1300,81.1000,80.0700,74.1500


## Constants

In [4]:
NUM_STOCKS = len(df)
print(f'Number of stocks in DataFrame: {NUM_STOCKS}')

Number of stocks in DataFrame: 48


In [5]:
ORIGINAL_NUM_DAYS = len(df.iloc[0])
print(f'Original number of days for which we have price data for each ticker: {ORIGINAL_NUM_DAYS}')

Original number of days for which we have price data for each ticker: 951


In [6]:
DAYS_PER_BATCH = 50
print(f'Numbers of days for which we have price data in one division of the original data: {DAYS_PER_BATCH}')

Numbers of days for which we have price data in one division of the original data: 50


## Creating the new DataFrame

In [7]:
# Save the tickers before dropping it
tickers = df['ticker']
df = df.drop('ticker', axis = 1)

# Now that the dataframe is numerical only, we can easily compute the following
df['mean'] = df.mean(axis=1)
df['std'] = df.std(axis=1)
df['min'] = df.min(axis=1)
df['max'] = df.max(axis=1)
df['ticker'] = tickers
df.head()

,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,...,day_946,day_947,day_948,day_949,day_950,mean,std,min,max,ticker
0,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,49.8548,50.01,49.2164,48.3968,...,49.7021,48.9735,48.7938,48.2923,47.4123,43.041980,5.792453,5.792453,53.2909,NVO
1,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,123.0300,123.89,120.8000,119.0100,...,172.0500,170.2900,169.5500,169.2400,167.6900,137.722917,38.664037,38.664037,194.6700,VRTX
2,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,531.7600,519.38,499.0000,510.5000,...,287.7000,280.4500,275.3000,277.2700,275.3100,381.977832,53.964763,53.964763,550.5400,REGN
3,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,43.8800,42.07,41.8400,39.4200,...,73.8200,76.7000,79.1700,85.6000,85.2600,58.427116,12.564510,12.564510,85.6000,SGEN
4,87.2400,91.2900,90.1600,92.4900,93.4800,94.1800,92.1800,93.49,92.7600,87.3900,...,83.6300,80.1300,81.1000,80.0700,74.1500,80.599735,25.244504,25.244504,147.8900,ALNY


In [8]:
new_col_names = ['ticker', 'std', 'min', 'max', 'mean']

for i in range(1, DAYS_PER_BATCH + 1, 1):
    col_name = 'price_' + str(i)
    new_col_names.append(col_name)

In [9]:
# Create an empty DataFrame with these column headings
new = pd.DataFrame(columns=new_col_names)
new.head()

,ticker,std,min,max,mean,price_1,price_2,price_3,price_4,price_5,...,price_41,price_42,price_43,price_44,price_45,price_46,price_47,price_48,price_49,price_50


In [10]:
for i in range(1, DAYS_PER_BATCH + 1, 1):
    dfs = []
    original_cols = [f'day_{j}' for j in range(i, ORIGINAL_NUM_DAYS, 50)]
    for name in original_cols:
        dfs.append(df[name])

    combined = pd.concat(dfs, ignore_index=True).reindex()
    new[f'price_{i}'] = combined

In [11]:
new.head()

,ticker,std,min,max,mean,price_1,price_2,price_3,price_4,price_5,...,price_41,price_42,price_43,price_44,price_45,price_46,price_47,price_48,price_49,price_50
0,NaN,NaN,NaN,NaN,NaN,49.5701,49.7771,50.2947,50.4069,50.6312,...,45.1445,43.5744,44.6355,45.0323,44.8253,45.1445,45.4723,44.6527,48.6384,48.5348
1,NaN,NaN,NaN,NaN,NaN,123.9200,124.3800,123.9900,124.9700,127.6800,...,88.0600,84.1200,87.3300,86.4400,86.8300,86.1300,88.9400,92.2500,89.9100,87.3600
2,NaN,NaN,NaN,NaN,NaN,531.0600,535.5600,536.4500,542.2800,550.5400,...,392.0000,383.6900,396.3600,396.9200,396.2400,387.8600,410.0000,406.2300,404.5500,401.7900
3,NaN,NaN,NaN,NaN,NaN,42.7300,43.7100,43.7000,44.5100,45.3000,...,30.5300,29.2000,30.1200,30.4500,30.9500,30.2800,31.8600,32.5300,31.7300,32.4600
4,NaN,NaN,NaN,NaN,NaN,87.2400,91.2900,90.1600,92.4900,93.4800,...,62.0500,58.7400,58.9600,57.7200,58.0000,59.4900,59.9200,62.3300,60.6600,58.9900


In [12]:
NUM_ENTRIES = len(new)
print(NUM_ENTRIES)

912


In [13]:
means = stds = min_vals = max_vals = tickers = np.zeros(NUM_ENTRIES)
tickers = list(map(str, tickers))

"""
The new dataframe consists of 912 rows. They are ordered in 19 sets of 48 each. 
This means that the 0th value and 48th value both point to the same stock
"""

for i in range(NUM_STOCKS):
    row = df.iloc[i]
    mean = row['mean']
    std = row['std']
    min_val = row['min']
    max_val = row['max']
    ticker = row['ticker']


In [14]:
for i in range(0, 48, 1):
    row = df.iloc[i]
    mean = row['mean']
    std = row['std']
    min_val = row['min']
    max_val = row['max']
    ticker = row['ticker']
    for j in range(i, 912, 48):
        means[j] = mean
        stds[j] = std
        min_vals[j] = min_val
        max_vals[j] = max_val
        tickers[j] = ticker

In [15]:
new['mean'] = means
new['std'] = stds
new['min'] = min_vals
new['max'] = max_vals
new['ticker'] = tickers

In [16]:
new.head(50)

,ticker,std,min,max,mean,price_1,price_2,price_3,price_4,price_5,...,price_41,price_42,price_43,price_44,price_45,price_46,price_47,price_48,price_49,price_50
0,NVO,53.2909,53.2909,53.2909,53.2909,49.5701,49.7771,50.2947,50.4069,50.6312,...,45.1445,43.5744,44.6355,45.0323,44.8253,45.1445,45.4723,44.6527,48.6384,48.5348
1,VRTX,194.6700,194.6700,194.6700,194.6700,123.9200,124.3800,123.9900,124.9700,127.6800,...,88.0600,84.1200,87.3300,86.4400,86.8300,86.1300,88.9400,92.2500,89.9100,87.3600
2,REGN,550.5400,550.5400,550.5400,550.5400,531.0600,535.5600,536.4500,542.2800,550.5400,...,392.0000,383.6900,396.3600,396.9200,396.2400,387.8600,410.0000,406.2300,404.5500,401.7900
3,SGEN,85.6000,85.6000,85.6000,85.6000,42.7300,43.7100,43.7000,44.5100,45.3000,...,30.5300,29.2000,30.1200,30.4500,30.9500,30.2800,31.8600,32.5300,31.7300,32.4600
4,ALNY,147.8900,147.8900,147.8900,147.8900,87.2400,91.2900,90.1600,92.4900,93.4800,...,62.0500,58.7400,58.9600,57.7200,58.0000,59.4900,59.9200,62.3300,60.6600,58.9900
5,BMRN,106.3000,106.3000,106.3000,106.3000,104.4000,105.8300,105.9100,106.0200,106.3000,...,78.1100,74.3100,76.9200,76.6400,80.7500,82.5000,87.4400,88.9100,89.7300,87.7000
6,INCY,151.6900,151.6900,151.6900,151.6900,109.2100,110.2600,108.9800,109.9800,110.2700,...,73.4700,71.2900,74.0000,75.0000,74.7600,74.0100,74.7900,74.0000,73.5100,71.6400
7,TECH,212.8470,212.8470,212.8470,212.8470,85.2154,85.3011,86.6815,87.7763,87.4241,...,84.3325,82.0582,83.0329,83.6540,82.9182,82.3544,85.6226,86.1291,87.2471,87.8683
8,UTHR,169.3100,169.3100,169.3100,169.3100,159.2800,159.1500,159.9900,161.8700,159.2600,...,128.2000,126.2600,128.7500,125.8300,123.0100,122.1400,126.9400,128.4800,126.0000,127.0000
9,JAZZ,183.4900,183.4900,183.4900,183.4900,144.1500,144.2400,141.7800,142.6500,142.5600,...,124.9600,120.0000,121.9200,124.7200,123.4800,121.4200,123.4000,124.9200,125.4400,122.7800


As we can see, we have the same metadata for rows 0 and 1 as we do for 48 and 49 respectively. 

## Preparing data for ML

### Establishing variables

We first need to establish input feature names and the name of the output feature. Here, we'll use every feature apart from the 50th day's price as the target feature, i.e, the 50th day's price is our y-variable.

In [17]:
inp_features = new_col_names[1:-1]
X = new[inp_features]
X.head()

,std,min,max,mean,price_1,price_2,price_3,price_4,price_5,price_6,...,price_40,price_41,price_42,price_43,price_44,price_45,price_46,price_47,price_48,price_49
0,53.2909,53.2909,53.2909,53.2909,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,...,44.1869,45.1445,43.5744,44.6355,45.0323,44.8253,45.1445,45.4723,44.6527,48.6384
1,194.6700,194.6700,194.6700,194.6700,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,...,89.4100,88.0600,84.1200,87.3300,86.4400,86.8300,86.1300,88.9400,92.2500,89.9100
2,550.5400,550.5400,550.5400,550.5400,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,...,399.0500,392.0000,383.6900,396.3600,396.9200,396.2400,387.8600,410.0000,406.2300,404.5500
3,85.6000,85.6000,85.6000,85.6000,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,...,30.7100,30.5300,29.2000,30.1200,30.4500,30.9500,30.2800,31.8600,32.5300,31.7300
4,147.8900,147.8900,147.8900,147.8900,87.2400,91.2900,90.1600,92.4900,93.4800,94.1800,...,64.5200,62.0500,58.7400,58.9600,57.7200,58.0000,59.4900,59.9200,62.3300,60.6600


In [18]:
target_feature = new_col_names[-1]
y = new[target_feature]
y

0       48.5348
1       87.3600
2      401.7900
3       32.4600
4       58.9900
         ...   
907     51.4146
908     13.0000
909      9.0800
910     15.7900
911     10.3984
Name: price_50, Length: 912, dtype: float64

### Training-Testing Split
For this notebook, we're just splitting data into training and testing. You might want to create a validation set as well.

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Let's save this in case we want to come back later

In [20]:
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

# Creating the model

## Random forest model

We will be using a simple random forest model for demonstation purposes in this notebook. **You should not use this simple of a model in the competition**. As you will see, it will not achieve ideal performance. This will put your models at a competitive disadvantage when predicting prices.

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [49]:
model = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=10)
model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=10, random_state=0)

In [50]:
y_pred = model.predict(X_test)

In [51]:
X_test

,std,min,max,mean,price_1,price_2,price_3,price_4,price_5,price_6,...,price_40,price_41,price_42,price_43,price_44,price_45,price_46,price_47,price_48,price_49
191,315.52,315.52,315.52,315.52,62.64,68.00,68.96,65.44,66.40,66.40,...,71.8864,71.3952,70.24,71.20,71.84,72.48,69.60,71.52,72.00,72.80
832,66.25,66.25,66.25,66.25,57.49,55.19,57.00,58.19,60.50,58.00,...,53.8700,56.3200,56.48,55.98,57.28,58.15,55.66,56.59,57.52,55.59
111,131.93,131.93,131.93,131.93,14.67,14.87,13.96,14.00,14.06,13.96,...,12.6111,12.8600,12.94,12.84,12.90,13.21,13.24,12.85,12.89,13.00
262,116.35,116.35,116.35,116.35,72.08,73.11,72.48,71.22,74.27,74.02,...,77.5800,78.2500,77.23,77.00,78.29,75.90,77.55,84.65,86.93,89.00
670,31.26,31.26,31.26,31.26,18.62,18.55,19.10,18.52,18.40,18.77,...,18.1000,18.1300,18.45,18.36,18.20,18.35,18.58,18.72,18.90,18.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,85.60,85.60,85.60,85.60,68.62,67.04,67.21,67.43,67.24,68.00,...,68.7500,67.8000,63.64,62.90,63.92,61.62,60.52,61.54,62.74,62.23
121,44.00,44.00,44.00,44.00,31.44,32.27,32.98,33.98,34.53,35.28,...,33.3100,32.7200,33.00,33.16,33.18,35.60,36.35,36.63,36.13,36.92
772,147.89,147.89,147.89,147.89,85.71,86.05,88.16,85.46,85.15,82.77,...,87.9500,86.9800,88.50,91.89,84.25,79.80,75.05,73.44,70.57,69.40
108,32.20,32.20,32.20,32.20,5.24,5.35,5.39,5.81,6.10,6.20,...,8.1100,8.2900,8.36,8.35,8.36,8.56,8.67,8.68,8.69,8.66


In [52]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1.4751459180327833
Mean Squared Error: 7.423541472927261
Root Mean Squared Error: 2.7246176746338673


## Custom Evaluation functions
These will help us better contextualise our predictions in the context of the dataset

MAPE: https://en.wikipedia.org/wiki/Mean_absolute_percentage_error

WMAPE: https://en.wikipedia.org/wiki/WMAPE

In [53]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

In [54]:
mape(y_test, y_pred)

0.030365772004431486

In [55]:
wmape(y_test, y_pred)

0.025936339222233453

# What's next?
Your job: Create a model that does the best possible job of predicting the next day's stock price based on historical data for that stock.
Explore:
- Creating training data in a different fashion
- New features
- Alternative algorithsm and models, such as neural networks
- Changing the hyperparameters on the random forest

**Remember what your algorithm will be fed and craft input features around that.**
Your algorithm will be given all-time historical data for each stock (as was the case in the original DataFrame here).  From there, you can choose how many days' data to use, what statistical features to extract, etc.

**Good luck!**

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=86376c80-cc11-49b3-a94a-d3d931acdd32' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

## Saving the model

In [56]:
import pickle

In [57]:
# Save the model
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

## Load the model

In [58]:
model = pickle.load(open(filename, 'rb'))
model

RandomForestRegressor(n_jobs=10, random_state=0)